# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import numpy as np
import gmaps
import os
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,23.52,59,100,11.86,PN,1711834859
1,1,tayu,-6.5397,111.0518,25.19,90,19,3.13,ID,1711835159
2,2,waitangi,-43.9535,-176.5597,15.46,97,100,3.58,NZ,1711835159
3,3,severo-yeniseyskiy,60.3725,93.0408,-1.90,95,100,2.68,RU,1711835160
4,4,edinburgh of the seven seas,-37.0676,-12.3116,14.42,62,89,0.57,SH,1711835160


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points("Lng", 
                                      "Lat", 
                                      geo = True,
                                      color = "City",
                                      alpha = 0.6,
                                      size = "Humidity",
                                      tiles = "OSM"
                                     )

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
# Criteria set:
# A max temperature lower than 27 degrees but higher than 21
# Cloudiness equal 0
# Wind speed less than 4.5 m/s


weather_filtered_df = city_data_df.loc[(city_data_df["Max Temp"] < 27)
                                       & (city_data_df["Max Temp"] > 21) 
                                       & (city_data_df["Cloudiness"] == 0)
                                       & (city_data_df["Wind Speed"] < 4.5)
                                      ]

# Drop any rows with null values
weather_filtered_df.dropna()

# Display sample data
weather_filtered_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
116,116,port hedland,-20.3167,118.5667,26.03,89,0,2.06,AU,1711835202
169,169,siwa oasis,29.2041,25.5195,21.83,31,0,0.79,EG,1711835220
179,179,la foa,-21.7108,165.8276,24.49,61,0,1.06,NC,1711835224
238,238,dayrut,27.5560,30.8076,23.00,33,0,4.31,EG,1711835247
276,276,zayed city,24.3768,54.5890,25.00,83,0,2.06,AE,1711835260
282,282,illapel,-31.6308,-71.1653,25.77,45,0,2.39,CL,1711835172
375,375,punta del este,-34.9667,-54.9500,26.06,83,0,1.03,UY,1711835295
399,399,dom pedrito,-30.9828,-54.6731,23.61,59,0,1.11,BR,1711835304
404,404,rio grande,-32.0350,-52.0986,26.06,82,0,3.42,BR,1711835076
406,406,dalton,34.7698,-84.9702,24.82,33,0,3.09,US,1711835008


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = weather_filtered_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
116,port hedland,AU,-20.3167,118.5667,89,
169,siwa oasis,EG,29.2041,25.5195,31,
179,la foa,NC,-21.7108,165.8276,61,
238,dayrut,EG,27.5560,30.8076,33,
276,zayed city,AE,24.3768,54.5890,83,
282,illapel,CL,-31.6308,-71.1653,45,
375,punta del este,UY,-34.9667,-54.9500,83,
399,dom pedrito,BR,-30.9828,-54.6731,59,
404,rio grande,BR,-32.0350,-52.0986,82,
406,dalton,US,34.7698,-84.9702,33,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
# Set parameters to search for a hotel
radius = 10000 
params = {"categories": "accommodation.hotel",
          "apiKey":geoapify_key
         }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]

    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =  f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
port hedland - nearest hotel: The Esplanade Hotel
siwa oasis - nearest hotel: فندق الكيلانى
la foa - nearest hotel: Oasis de Tendea
dayrut - nearest hotel: No hotel found
zayed city - nearest hotel: L'Arabia hotel
illapel - nearest hotel: Domingo Ortiz de Rosas
punta del este - nearest hotel: Hotel Aquarium
dom pedrito - nearest hotel: Hotel Alexandre
rio grande - nearest hotel: Hotel Vila Moura Executivo
dalton - nearest hotel: Days Inn
graaff-reinet - nearest hotel: Profcon Resort
atar - nearest hotel: Residence Egueil


,City,Country,Lat,Lng,Humidity,Hotel Name
116,port hedland,AU,-20.3167,118.5667,89,The Esplanade Hotel
169,siwa oasis,EG,29.2041,25.5195,31,فندق الكيلانى
179,la foa,NC,-21.7108,165.8276,61,Oasis de Tendea
238,dayrut,EG,27.5560,30.8076,33,No hotel found
276,zayed city,AE,24.3768,54.5890,83,L'Arabia hotel
282,illapel,CL,-31.6308,-71.1653,45,Domingo Ortiz de Rosas
375,punta del este,UY,-34.9667,-54.9500,83,Hotel Aquarium
399,dom pedrito,BR,-30.9828,-54.6731,59,Hotel Alexandre
404,rio grande,BR,-32.0350,-52.0986,82,Hotel Vila Moura Executivo
406,dalton,US,34.7698,-84.9702,33,Days Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.8,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"]
                                  )

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)